# Training Models

### Hypothesis:

1. Optimizador rutas --> Ruben

In [ ]:
#Load Data

data_trips_red_inferences = pd.read_csv("../data/processed/data_trips_red_inferences.csv", sep=",")

In [ ]:
#Factorization

data_trips_red_inferences = data_trips_red_inferences.apply(lambda x: pd.factorize(x)[0])

Creación de nuevo dataframe de apoyo para el cálculo de rutas (sobre el que va a trabajar el optimizador de rutas).

In [ ]:

# Selecciona las columnas relevantes para las rutas
columns_of_interest = [
    'latitude_unlock', 'longitude_unlock',
    'latitude_lock', 'longitude_lock',
    'distance', 'station_unlock_count', 'station_lock_count',
    'trip_speed', 'lock_unlock_time_gap_hours'
]

# Crea un nuevo DataFrame de rutas
routes_df = data_trips_red_inferences[columns_of_interest].copy()

In [ ]:
# Guarda el DataFrame en un archivo CSV
routes_df.to_csv("rutas_dataset.csv", index=False)

In [ ]:
# Cargar el modelo de regresión utilizando pickle
with open("../models/regression_model_linear.pkl", "rb") as model_file:
    linear_model = pickle.load(model_file)

Método 1👉Vectorizador.

In [ ]:
# Realizar predicciones de duración del viaje para las nuevas rutas
predicted_durations = linear_model.predict(routes_df)

# Añadir las predicciones al DataFrame de rutas
routes_df['predicted_duration'] = predicted_durations

# Mostrar el DataFrame con las predicciones
print(routes_df)	

En este bloque, el método predict de la biblioteca scikit-learn se aplica directamente a todo el DataFrame routes_df. Este método de aplicación directa de operaciones a matrices (o DataFrames) se conoce como operación vectorizada. La eficiencia radica en el hecho de que las operaciones vectorizadas se implementan internamente utilizando código altamente optimizado en lenguajes de bajo nivel como C o Fortran, lo que las hace mucho más rápidas que las operaciones de bucle tradicionales de Python.

Método 2 👉 Iterador

In [ ]:
 # Ya se ha cargado el modelo de regresión y tenemos routes_df con las predicciones

def calcular_duracion_viaje(row):
    # Esta función toma una fila del DataFrame y calcula la duración del viaje basándose en las predicciones del modelo
    # Asegúrate de que las columnas usadas coincidan con las que se utilizaron durante el entrenamiento del modelo
    features = row[['latitude_unlock', 'longitude_unlock', 'latitude_lock', 'longitude_lock',
                    'distance', 'station_unlock_count', 'station_lock_count',
                    'trip_speed', 'lock_unlock_time_gap_hours']]
    
    # Realiza la predicción de duración del viaje
    predicted_duration = linear_model.predict([features])[0]
    
    return predicted_duration

# Aplica la función calcular_duracion_viaje a cada fila del DataFrame
routes_df['predicted_duration'] = routes_df.apply(calcular_duracion_viaje, axis=1)

# Muestra el DataFrame con las predicciones actualizadas
print(routes_df)

En este bloque, la función calcular_duracion_viaje se aplica a cada fila del DataFrame utilizando apply con el argumento axis=1 para especificar que la aplicación se realiza a lo largo de las filas. Este enfoque es más iterativo y se ejecuta en un bucle implícito, donde cada fila del DataFrame se pasa individualmente a la función.

Comparación:

Eficiencia:

El primer bloque es más eficiente en términos de rendimiento debido a la vectorización. Las operaciones vectorizadas en NumPy y scikit-learn están altamente optimizadas y ejecutadas en código compilado, lo que las hace más rápidas.
El segundo bloque es menos eficiente, ya que implica la iteración a través de las filas y la aplicación de la función a cada fila por separado, lo cual puede ser más lento.

Flexibilidad:

El segundo bloque puede proporcionar más flexibilidad porque puedes realizar operaciones más complejas en cada fila antes de hacer la predicción. Puedes realizar cualquier lógica específica requerida para tus datos antes de aplicar la predicción.
El primer bloque es más directo y puede ser más adecuado cuando las operaciones en cada fila son simples y no requieren lógica adicional.

2. Predicción de demanda por estaciones --> Ruben

3. Gamificación --> Juan
    *  3.1 Consumo de calorías
    *  3.2 Contribución a la huella de carbono
    *  3.3 Optimizador rutas